In [27]:
pip install cinemagoer

In [89]:
import pandas as pd
import requests
import re
from imdb import Cinemagoer
from bs4 import BeautifulSoup
import time
import numpy as np

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
df=pd.read_csv("/content/drive/MyDrive/Project/Final_Movie_Dataset.csv", index_col=0)

In [31]:
def extract_ids(link_string):
    # Use regex to find all occurrences of 'tt' followed by digits, then remove 'tt'
    ids = re.findall(r'tt(\d+)', link_string)  # Capture only the digits after 'tt'
    # Join the list of numeric IDs into a single string, separated by commas
    return ', '.join(ids)

In [32]:
df['movie_id2'] = df['movie_id'].apply(extract_ids)

In [33]:
df

,movie_id,movie_name,year,genre,overview,director,cast,movie_id2
0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...",15354916
1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...",15748830
2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...",11663228
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...",14993250
4,tt15732324,OMG 2,2023,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...",15732324
...,...,...,...,...,...,...,...,...
4026,tt33808098,Cycle Mahesh,2024.0,unknown,A young worker's epic bicycle journey home and...,Suhel Banerjee,unknown,33808098
4027,tt33808618,Maryade Prashne,2024.0,unknown,Unknown,Nagaraja Somayaji,"Poornachandra Mysuru, Rekha Kudligi, Teju Bela...",33808618
4028,tt33810459,Uruttu Factory,2024.0,Documentary,Unknown,"Sid, Sid",Sid,33810459
4029,tt33812085,Satchi Perumal (Witness Perumal),2024.0,Drama,'Satchi Perumal' with the main lead Perumal fr...,RP. Vinnu,"Pandiamma, Verra Paramasivam, V P. Rajasekar, ...",33812085


In [40]:
ia = Cinemagoer()

In [41]:
keywords_list=[]

In [42]:
df.shape

(4031, 8)

In [43]:
for movie_name in df['movie_name']:  # Replace 'movie_name' with the actual column name in your CSV
    # Search for the movie by title
    movies = ia.search_keyword(movie_name)

    if movies:
        # Get the first search result
        # movie_titles = [movie['title'] for movie in movies]
        keywords_list.append(', '.join(movies))  # Join movie titles as a string

    else:
        keywords_list.append('unknown')  # If movie not found
    time.sleep(0.3)

In [44]:
# keyword = pd.DataFrame({'Keyword':keywords_list})
keyword = pd.DataFrame(keywords_list)

In [45]:
len(keywords_list)

4031

In [46]:
df = pd.concat([df, keyword], axis=1)

In [77]:
df.head(5)

,movie_id,movie_name,year,genre,overview,director,cast,movie_id2,keyword
0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...",15354916,"amar jawan jyoti, trap jaw ant"
1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...",15748830,jaane na dena
2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...",11663228,"jailer, trailer, corrupt jailer, cruel jailer,..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...",14993250,unknown
4,tt15732324,OMG 2,2023,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...",15732324,unknown


In [57]:
df.rename(columns={0: 'keyword'}, inplace=True)

In [60]:
df.to_csv('final_movie_dataset_with_keyword.csv')

In [94]:
# List of IMDb keywords page URLs for 10 different movies
imdb_ids = df['movie_id'].tolist()
base_url = 'https://www.imdb.com/title/{}/keywords/'

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive'
}

In [95]:
# Function to scrape keywords from a given IMDb URL
def scrape_keywords(url):
    # Send a GET request to the page with headers
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the keywords
    keywords = soup.find_all('a', class_='ipc-metadata-list-summary-item__t')

    # Extract text for each keyword and store in a list
    keyword_list = [keyword.get_text(strip=True) for keyword in keywords]

    # Use join() to combine all keywords into a single string separated by commas
    keywords_string = ', '.join(keyword_list)

    return keywords_string


In [96]:
# List to store data for each movie
data = []

# Loop through each IMDb ID, dynamically create the URL, and collect keywords
for i, imdb_id in enumerate(imdb_ids):
    # Dynamically create the URL by inserting the IMDb ID
    url = base_url.format(imdb_id)

    # Scrape the keywords for the movie
    keywords = scrape_keywords(url)

    # Append movie data as a dictionary (You can use movie titles from your dataset instead of 'Movie {i + 1}')
    data.append({
        # 'Movie': df['title'].iloc[i],  # Assuming you have a 'title' column in your dataset for movie titles
        'Keywords': keywords
    })

In [97]:
# Create a DataFrame
key_df = pd.DataFrame(data)


,Keywords
0,"revenge, chase, suicide, farmer, train, injust..."
1,"single mother, police, investigation, criminal..."
2,"prison, ex cop, intermission, action hero, one..."
3,"love, family relationships, rom com, news repo..."
4,"courtroom, sex education, court room drama, re..."
...,...
4026,
4027,
4028,
4029,


In [103]:
key_df.sample(5)

,Keywords
1908,love
3305,
2559,
219,train
1308,"india, based on real people, docudrama drama, ..."


In [104]:
new_df = pd.concat([df, key_df], axis=1)

In [107]:
new_df.drop(['keyword'], axis=1, inplace=True)

In [108]:
new_df

,movie_id,movie_name,year,genre,overview,director,cast,movie_id2,Keywords
0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...",15354916,"revenge, chase, suicide, farmer, train, injust..."
1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...",15748830,"single mother, police, investigation, criminal..."
2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...",11663228,"prison, ex cop, intermission, action hero, one..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...",14993250,"love, family relationships, rom com, news repo..."
4,tt15732324,OMG 2,2023,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...",15732324,"courtroom, sex education, court room drama, re..."
...,...,...,...,...,...,...,...,...,...
4026,tt33808098,Cycle Mahesh,2024.0,unknown,A young worker's epic bicycle journey home and...,Suhel Banerjee,unknown,33808098,
4027,tt33808618,Maryade Prashne,2024.0,unknown,Unknown,Nagaraja Somayaji,"Poornachandra Mysuru, Rekha Kudligi, Teju Bela...",33808618,
4028,tt33810459,Uruttu Factory,2024.0,Documentary,Unknown,"Sid, Sid",Sid,33810459,
4029,tt33812085,Satchi Perumal (Witness Perumal),2024.0,Drama,'Satchi Perumal' with the main lead Perumal fr...,RP. Vinnu,"Pandiamma, Verra Paramasivam, V P. Rajasekar, ...",33812085,


In [110]:
new_df.to_csv('/content/drive/MyDrive/Project/final_movie_dataset_with_keyword.csv')